# Training & inference notebook
Credit to [Tarun Mishra](https://www.kaggle.com/tarundirector) – this code is heavily based on his [notebook](https://www.kaggle.com/code/tarundirector/sensor-pulse-viz-eda-for-bfrb-detection?scriptVersionId=243465321).

## Setup

### imports

#### Training imports

In [1]:
import os
import json
from glob import glob
from functools import partial
from datetime import datetime
from itertools import pairwise
from os.path import join, realpath
from typing import Optional, Literal

import torch
import kagglehub
import numpy as np
import pandas as pd
import plotly.express as px
from torch import nn, Tensor
from pandas import DataFrame as DF
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader as DL
from rich.progress import Progress, Task, track
from torch.optim.lr_scheduler import ConstantLR, LRScheduler
metric_package = kagglehub.package_import('wasupandceacar/cmi-metric', bypass_confirmation=True)

#### inference imports

In [2]:
import os
import json
import warnings
from os.path import join
from tqdm.notebook import tqdm
from itertools import pairwise, product

import torch
import numpy as np
import pandas as pd
import polars as pl
from numpy import ndarray
from torch import nn, Tensor
from numpy.linalg import norm
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
# from kagglehub import competition_download, dataset_download, model_download
import kagglehub
metric_package = kagglehub.package_import('wasupandceacar/cmi-metric', bypass_confirmation=True)

import kaggle_evaluation.cmi_inference_server

#### kaggle notbook Imports

In [3]:
import random
import numpy as np
import torch
import os

def seed_everything(seed=42):
    """Set all random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.use_deterministic_algorithms(True, warn_only=True)

SEED = 42
seed_everything(seed=SEED)

import pandas as pd
import polars as pl
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.utils.class_weight import compute_class_weight
import joblib
from tqdm import tqdm

from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from tensorflow.keras.preprocessing.sequence import pad_sequences as keras_pad_sequences

import kaggle_evaluation.cmi_inference_server
from matplotlib import pyplot as plt

### Configs

#### Training config

In [25]:
NB_CROSS_VALIDATIONS = 5
TRAINING_EPOCHS = 50
STARTING_LR = 0.0005
TRAIN_BATCH_SIZE = 256
VALIDATION_BATCH_SIZE = 4 * TRAIN_BATCH_SIZE
TARGET_NAMES = sorted([
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
])
TARGET_NAMES_NDARRAY = np.asarray(TARGET_NAMES)
MOCK_TRAINING_EPOCHS = 15
MOCK_TRAINING_GAMMA = 1.01
MAX_LR_TO_MIN_DIV_FACTOR = 10

COMPETITION_HANDLE = "cmi-detect-behavior-with-sensor-data"

#### Inference config 

In [5]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
GRAVITY_WORLD = np.array([0, 0, 9.81], "float32")
RAW_ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
LINEAR_ACC_COLS = ["gravity_free_" + col for col in RAW_ACCELRATION_COLS]
COMPETITION_HANDLE = "cmi-detect-behavior-with-sensor-data"
CATEGORY_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
META_DATA_COLUMNS = [
    'row_id',
    'sequence_type',
    'sequence_id',
    'sequence_counter',
    'subject',
    'orientation',
    'behavior',
    'phase',
    'gesture',
]
DATASET_DF_DTYPES = {
    "acc_x": "float32", "acc_y": "float32", "acc_z": "float32",
    "thm_1":"float32", "thm_2":"float32", "thm_3":"float32", "thm_4":"float32", "thm_5":"float32",
    "sequence_counter": "int32",
    **{col: "category" for col in CATEGORY_COLUMNS},
    **{f"tof_{i_1}_v{i_2}": "float32" for i_1, i_2 in product(range(1, 5), range(64))},
}
PREPROCESSED_DATASET_HANDLE = "mauroabidalcarrer/prepocessed-cmi-2025"
# The quantile of the sequences len used to pad/truncate during preprocessing
SEQUENCE_NORMED_LEN_QUANTILE = 0.95
# SAMPLING_FREQUENCY = 10 #Hz
N_FOLDS = 5
VALIDATION_FRACTION = 0.2
TARGET_NAMES = sorted([
    "Above ear - pull hair",
    "Cheek - pinch skin",
    "Eyebrow - pull hair",
    "Eyelash - pull hair",
    "Feel around in tray and pull out an object",
    "Forehead - pull hairline",
    "Forehead - scratch",
    "Neck - pinch skin",
    "Neck - scratch",
    "Text on phone",
    "Wave hello",
    "Write name in air",
    "Write name on leg",
    "Drink from bottle/cup",
    "Pinch knee/leg skin",
    "Pull air toward your face",
    "Scratch knee/leg skin",
    "Glasses on/off"
])
EPSILON=1e-8
DELTA_ROTATION_ANGULAR_VELOCITY_COLS = ["angular_vel_x", "angular_vel_y", "angular_vel_z"]
DELTA_ROTATION_AXES_COLS = ["rotation_axis_x", "rotation_axis_y", "rotation_axis_z"]
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [6]:
def get_feature_cols(df:DF) -> list[str]:
    return sorted(list(set(df.columns) - set(META_DATA_COLUMNS) - set(TARGET_NAMES)))

### Supress performance warngings

In [7]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### device setup

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Dataset Setup

In [9]:
dataset_path = kagglehub.dataset_download(
    handle="mauroabidalcarrer/prepocessed-cmi-2025",
)

class CMIDataset(TensorDataset):
    def __init__(
        self,
        parent_dir: str,
        split: Optional[Literal["train", "validation"]]=None,
        subset: Optional[int]=None,
        force_download=False
    ):
        dataset_path = kagglehub.dataset_download(
            handle="mauroabidalcarrer/prepocessed-cmi-2025",
            force_download=force_download
        )
        parent_dir = join(dataset_path, "preprocessed_dataset", parent_dir)
        split = "" if split is None else split + "_"
        x = np.load(join(parent_dir, f"{split}X.npy")).swapaxes(1, 2)
        y = np.load(join(parent_dir, f"{split}Y.npy"))
        if subset is not None:
            x = x[:subset]
            y = y[:subset]
        super().__init__(
            torch.from_numpy(x), 
            torch.from_numpy(y),
        )

In [10]:
meta_data_path = join(
    dataset_path,
    "preprocessed_dataset",
    "full_dataset_meta_data.json"
)
with open(meta_data_path, "r") as fp:
    meta_data = json.load(fp)
# Convert target names into a ndarray to index it batchwise.
# meta_data["target_names"] = np.asarray(meta_data["target_names"])
non_imu_feats_idx = [feat_idx for feat_idx, feat in enumerate(meta_data["feature_cols"]) if feat.startswith(("thm", "tof"))]
non_imu_feats = [feat for feat in meta_data["feature_cols"] if feat.startswith(("thm", "tof"))]
imu_feats_idx = [feat_idx for feat_idx, feat in enumerate(meta_data["feature_cols"]) if not feat.startswith(("thm", "tof"))]
imu_feats = [feat for feat in meta_data["feature_cols"] if not feat.startswith(("thm", "tof"))]

### BFRBs indices

In [11]:
print("Loading datasets...")
competition_dataset_path = kagglehub.competition_download(COMPETITION_HANDLE)
train_df = pd.read_csv(join(competition_dataset_path, "train.csv"))
train_dem_df = pd.read_csv(join(competition_dataset_path, "train_demographics.csv"))
test_df = pd.read_csv(join(competition_dataset_path, "test.csv"))
test_dem_df = pd.read_csv(join(competition_dataset_path, "test_demographics.csv"))
print(f"Train rows: {len(train_df)}, Test rows: {len(test_df)}")

# Encode labels
label_encoder = LabelEncoder()
train_df['gesture'] = label_encoder.fit_transform(train_df['gesture'].astype(str))
gesture_classes = label_encoder.classes_

bfrb_gestures = [
    'Above ear - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Neck - pinch skin',
    'Neck - scratch',
    'Cheek - pinch skin'
]
bfrb_indices = label_encoder.transform(bfrb_gestures)

Loading datasets...
Train rows: 574945, Test rows: 107


## Model definition

In [ ]:
class SqueezeExcitationBlock(nn.Module):
    # Copy/paste of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Model implementation
    def __init__(self, channels:int, reduction:int=8):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction, bias=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, L)
        se = F.adaptive_avg_pool1d(x, 1).squeeze(-1)      # -> (B, C)
        se = F.relu(self.fc1(se), inplace=True)          # -> (B, C//r)
        se = self.sigmoid(self.fc2(se)).unsqueeze(-1)    # -> (B, C, 1)
        return x * se

class ResidualBlock(nn.Module):
    def __init__(self, in_chns:int, out_chns:int, dropout_ratio:float=0.3):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Conv1d(in_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
            nn.ReLU(),
            nn.Conv1d(out_chns, out_chns, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_chns),
            SqueezeExcitationBlock(out_chns),
        )
        self.head = nn.Sequential(nn.ReLU(), nn.Dropout(dropout_ratio))
        if in_chns == out_chns:
            self.skip_connection = nn.Identity() 
        else:
            # TODO: set bias to False ?
            self.skip_connection = nn.Sequential(
                nn.Conv1d(in_chns, out_chns, 1),
                nn.BatchNorm1d(out_chns)
            )
            self.head.add_module("max_pool", nn.MaxPool1d(2))
            
    def forward(self, x:Tensor) -> Tensor:
        activaition_maps = self.skip_connection(x) + self.blocks(x)
        return self.head(activaition_maps)

# class MBConvBlock(nn.Module):
#     def __init__(self, in_chns:int, out_chns)

class Resnet(nn.Sequential):
    def __init__(
            self,
            in_channels:int,
            depth:int,
            # n_res_block_per_depth:int,
            mlp_width:int,
            n_class:int,
        ):
        chs_per_depth = [in_channels * 2 ** i for i in range(depth)]
        blocks_chns_it = pairwise(chs_per_depth)
        res_blocks = [ResidualBlock(in_chns, out_chns) for in_chns, out_chns in blocks_chns_it]
        # self.res_blocks = nn.ModuleList(self.res_blocks)
        super().__init__(
            *res_blocks,
            nn.Flatten(start_dim=1),
            nn.LazyLinear(mlp_width),
            nn.ReLU(),
            nn.Linear(mlp_width, n_class),
        )

    # def forward(self, x:Tensor) -> Tensor:
    #     activation_maps = x
    #     for res_block in self.res_blocks:
    #         activation_maps = nn.functional.max_pool1d(res_block(activation_maps), 2)
    #     # print("res block output shape:", activation_maps.shape)
    #     out = activation_maps.view(activation_maps.shape[0], -1)
    #     # print("flatten mlp head input shape:", out.shape)
    #     out = self.mlp_head(out)
    #     return out

### Create model function

In [14]:
def mk_model() -> nn.Module:
    nb_in_chans = len(meta_data["feature_cols"])
    return (
        Resnet(
            in_channels=nb_in_chans,
            depth=4,
            mlp_width=256,
            n_class=18
        )
        .to(device)
    )

print("input channels:", len(meta_data["feature_cols"]))

input channels: 66


## Training loop

In [15]:
def fit(epochs:int,
        model: nn.Module,
        scheduler: LRScheduler,
        optimizer: torch.optim.Optimizer,
        train_loader: DL,
        criterion: callable=nn.L1Loss(),
        evaluation_func: callable=None,
        validation_loader: DL=None,
        save_checkpoints=True,
    ) -> tuple[DF, str]:
    """
    Returns:
        (training_metrics, path_to_checkpoints)
    """
    # Setup
    metrics: list[dict] = []
    step = 0
    model_device = next(model.parameters()).device
    last_epoch_metric = {}
    # Training loop
    with Progress() as progress:
        task: Task = progress.add_task(
            "training...",
            total=len(train_loader),
        )
        for epoch in range(epochs):
            progress.update(
                task,
                description=f"epoch: {epoch}",
                completed=0,
            )
            total_epoch_loss = 0
            total_accuracy = 0
            for batch_idx, (x, y) in enumerate(train_loader):
                # forward
                x = x.to(model_device)
                y = y.to(model_device)
                model.train()
                optimizer.zero_grad()
                y_pred: Tensor = model(x)
                loss_value = criterion(y_pred, y)
                # Verify loss value
                if torch.isnan(loss_value).any().item():
                    progress.print("Warning: Got NaN loss, stopped training.")
                    return DF.from_records(metrics)
                if torch.isinf(loss_value).any().item():
                    progress.print("Warning: Got infinite loss, stopped training.")
                    return DF.from_records(metrics)
                # TODO: Use gradient clipping?
                loss_value.backward()
                optimizer.step()
                if step > 0: # If it's not the first training step, idk why it throws an error otherwise
                    scheduler.step()
                # metrics
                total_epoch_loss += loss_value.item()
                metrics.append({
                    "step": step,
                    "epoch": epoch,
                    "batch_train_loss": loss_value.item(),
                    "lr": optimizer.state_dict()["param_groups"][-1]["lr"],
                })
                step += 1
                if "validation_accuracy" in last_epoch_metric:
                    last_validation_acc = "%.2f" % last_epoch_metric["validation_accuracy"]
                    val_acc_str = "val. acc: " + last_validation_acc
                else:
                    val_acc_str = ""
                progress.update(
                    task,
                    advance=1,
                    description=f"epoch: {epoch}, batch_loss: {(total_epoch_loss / (batch_idx+1)):.2f}, {val_acc_str}"
                )
            # Post epoch evalution
            metrics[-1]["train_epoch_loss"] = total_epoch_loss / len(train_loader)
            metrics[-1]["train_epoch_accuracy"] = total_accuracy / len(train_loader)
            if evaluation_func:
                progress.update(
                    task,
                    completed=0,
                    description=f"epoch: {epoch}, evaluating..."
                )
                eval_metrics = evaluation_func(model, criterion, validation_loader)
                metrics[-1].update(eval_metrics)
            last_epoch_metric = metrics[-1]

    return DF.from_records(metrics)

### Create model and train model

In [16]:
def mk_model_and_fit(
        train_loader:DL,
        mk_scheduler:callable,
        epochs:int,
        validation_loader:Optional[DL]=None,
        save_checkpoints=False,
    ) -> tuple[nn.Module, DF, list[str]]:
    model = mk_model()
    optimizer = torch.optim.AdamW(model.parameters(), STARTING_LR)
    lr_scheduler = mk_scheduler(optimizer)
    training_metrics = fit(
        epochs=epochs,
        model=model,
        scheduler=lr_scheduler,
        optimizer=optimizer,
        train_loader=train_loader,
        criterion=nn.CrossEntropyLoss(),
        # evaluation_func=evaluate_model if validation_loader else None,
        validation_loader=validation_loader,
        save_checkpoints=save_checkpoints,
    )

    return model, training_metrics

## Search max learning rate

In [17]:
def post_process_mock_training_metrics(training_metrics:DF) -> DF:
    training_metrics = (
        training_metrics
        .query("batch_train_loss.notna()")
        .set_index("lr", drop=False)
        .sort_index()
    )
    training_metrics["ewm_batch_train_loss"] = (
        training_metrics
        .ewm(com=30, ignore_na=False)
        ["batch_train_loss"]
        .mean()
    )
    training_metrics["ewm_batch_train_loss_diff"] = training_metrics["ewm_batch_train_loss"].diff()
    return training_metrics

In [23]:
def plt_lr_search_training_metrics(training_metrics:DF):
    (    
        px.line(
            (
                training_metrics
                .reset_index(drop=True)
                .melt(
                    id_vars="lr",
                    value_vars=[
                        "batch_train_loss",
                        "ewm_batch_train_loss",
                        # "ewm_batch_train_loss_diff",
                    ],
                )
            ),
            x="lr",
            facet_row="variable",
            y="value",
            log_x=True,
            log_y=True,
            height=750,
        )
        .update_yaxes(matches=None)
        .show()
    )

In [26]:
full_dataset = CMIDataset("full_dataset")
full_dataset_loader = DL(full_dataset, TRAIN_BATCH_SIZE, shuffle=True)
_, mock_training_metrics = mk_model_and_fit(
    full_dataset_loader,
    partial(torch.optim.lr_scheduler.ExponentialLR, gamma=MOCK_TRAINING_GAMMA),
    MOCK_TRAINING_EPOCHS
)
mock_training_metrics = post_process_mock_training_metrics(mock_training_metrics)
plt_lr_search_training_metrics(mock_training_metrics)
max_lr = mock_training_metrics["ewm_batch_train_loss"].idxmin()
# max_lr = 0.002
print("Maximum learning rate:", max_lr)

Output()

Maximum learning rate: 0.007340758313789516


## Training loop

In [27]:
seed_everything(seed=SEED)

# criterion = soft_cross_entropy
criterion = torch.nn.CrossEntropyLoss()

n_splits = 5
batch_size = 128
gkf = GroupKFold(n_splits=n_splits)

fold_metrics = []
best_fold_metrics = []
best_models = []

fold_patterns = join(dataset_path, "preprocessed_dataset", "fold*")
fold_pths = glob(fold_patterns)[:NB_CROSS_VALIDATIONS]
all_training_metrics = {}


for fold, fold_pth in enumerate(fold_pths):
    print("training:", fold + 1)
    train_dataset = CMIDataset(fold_pth, "train")
    train_loader = DL(train_dataset, TRAIN_BATCH_SIZE, shuffle=True)
    validation_dataset = CMIDataset(fold_pth, "validation")
    validation_loader = DL(validation_dataset, VALIDATION_BATCH_SIZE, shuffle=False)
    print(f"\n{'='*50}")
    print(f"Fold {fold + 1}/{n_splits}")
    
    seed_everything(seed=SEED + fold)
    model = mk_model()
    
    # Optimizer et scheduler
    starting_lr = max_lr / 2
    optimizer = torch.optim.AdamW(model.parameters(), starting_lr)
    steps_per_epoch = len(train_loader)
    n_training_steps = TRAINING_EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=n_training_steps // 5,
        eta_min=starting_lr / 10,
    )

    # Early stopping
    best_metric = -np.inf
    best_binary_f1 = -np.inf
    best_macro_f1 = -np.inf
    patience = 15
    epochs_no_improve = 0
    
    for epoch in range(1, TRAINING_EPOCHS + 1):
        # Training phase
        model.train()
        train_loss = 0.0
        total = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device).clone()
            batch_y = batch_y.to(device)
            batch_x = batch_x.float()
    
            # TODO: Apply mixup?
    
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            scheduler.step()
    
            train_loss += loss.item() * batch_x.size(0)
            total += batch_x.size(0)
        train_loss /= total
    
        # Validation phase
        model.eval()
        val_loss = 0.0
        total = 0
        all_true = []
        all_pred = []
        
        with torch.no_grad():
            for batch_x, batch_y in validation_loader:
                batch_x = batch_x.to(device).clone()
                batch_y = batch_y.to(device)
                batch_x[VALIDATION_BATCH_SIZE // 2, non_imu_feats_idx] = 0.0
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item() * batch_x.size(0)
                total += batch_x.size(0)
                
                # Get predicted class indices
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                # Get true class indices from one-hot
                trues = torch.argmax(batch_y, dim=1).cpu().numpy()
                
                all_true.append(trues)
                all_pred.append(preds)
        
        val_loss /= total
        all_true = np.concatenate(all_true)
        all_pred = np.concatenate(all_pred)
        
        # Compute competition metrics
        # Binary classification: BFRB (1) vs non-BFRB (0)
        binary_true = np.isin(all_true, bfrb_indices).astype(int)
        binary_pred = np.isin(all_pred, bfrb_indices).astype(int)
        binary_f1 = f1_score(binary_true, binary_pred)
        
        # Collapse non-BFRB gestures into a single class
        collapsed_true = np.where(
            np.isin(all_true, bfrb_indices),
            all_true,
            len(bfrb_gestures)  # Single non-BFRB class
        )
        collapsed_pred = np.where(
            np.isin(all_pred, bfrb_indices),
            all_pred,
            len(bfrb_gestures)  # Single non-BFRB class
        )
        
        # Macro F1 on collapsed classes
        macro_f1 = f1_score(collapsed_true, collapsed_pred, average='macro')
        final_metric = (binary_f1 + macro_f1) / 2
        
        print(f"Epoch {epoch:02d}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        print(f"  Binary F1 = {binary_f1:.4f}, Macro F1 = {macro_f1:.4f}, Final Metric = {final_metric:.4f}")
        
        if final_metric > best_metric:
            best_metric = final_metric
            best_binary_f1 = binary_f1
            best_macro_f1 = macro_f1
            epochs_no_improve = 0
            best_model_state = model.state_dict()
            print(f"  New best metric! Saving model...")
        else:
            epochs_no_improve += 1
            # if epochs_no_improve >= patience:
            #     print(f"Early stopping triggered at epoch {epoch}")
            #     model.load_state_dict(best_model_state)
            #     break

    torch.save(best_model_state, f"best_model_fold{fold}.pth")
    best_models.append(best_model_state)

    fold_metrics.append({
        'binary_f1': binary_f1,
        'macro_f1': macro_f1,
        'final_metric': final_metric
    })
    
    best_fold_metrics.append({
        'binary_f1': best_binary_f1,
        'macro_f1': best_macro_f1,
        'final_metric': best_metric
    })
    
    print(f"\nFold {fold + 1} completed.")
    print(f"Final validation metrics - Binary F1: {binary_f1:.4f}, Macro F1: {macro_f1:.4f}, Final: {final_metric:.4f}")
    print(f"Best validation metrics - Binary F1: {best_binary_f1:.4f}, Macro F1: {best_macro_f1:.4f}, Final: {best_metric:.4f}")

print("\n" + "="*50)
print("Cross-Validation Results")
print("="*50)

# Statistiques pour les meilleures métriques
best_binary_f1 = [m['binary_f1'] for m in best_fold_metrics]
best_macro_f1 = [m['macro_f1'] for m in best_fold_metrics]
best_metrics = [m['final_metric'] for m in best_fold_metrics]

print("\nBest Fold-wise Metrics:")
for i, (bf1, mf1, fm) in enumerate(zip(best_binary_f1, best_macro_f1, best_metrics)):
    print(f"Fold {i+1}: Binary F1 = {bf1:.4f}, Macro F1 = {mf1:.4f}, Final = {fm:.4f}")

print("\nGlobal Statistics (Best Metrics):")
print(f"Mean Best Final Metric: {np.mean(best_metrics):.4f} ± {np.std(best_metrics):.4f}")
print(f"Mean Best Binary F1: {np.mean(best_binary_f1):.4f} ± {np.std(best_binary_f1):.4f}")
print(f"Mean Best Macro F1: {np.mean(best_macro_f1):.4f} ± {np.std(best_macro_f1):.4f}")

training: 1

Fold 1/5
Epoch 01: Train Loss = 3.9989, Val Loss = 2.7004
  Binary F1 = 0.7160, Macro F1 = 0.0832, Final Metric = 0.3996
  New best metric! Saving model...
Epoch 02: Train Loss = 2.4792, Val Loss = 2.2858
  Binary F1 = 0.8147, Macro F1 = 0.1711, Final Metric = 0.4929
  New best metric! Saving model...
Epoch 03: Train Loss = 2.1189, Val Loss = 1.9592
  Binary F1 = 0.8864, Macro F1 = 0.2195, Final Metric = 0.5529
  New best metric! Saving model...
Epoch 04: Train Loss = 1.7912, Val Loss = 1.6958
  Binary F1 = 0.9311, Macro F1 = 0.3274, Final Metric = 0.6292
  New best metric! Saving model...
Epoch 05: Train Loss = 1.5265, Val Loss = 1.4968
  Binary F1 = 0.9487, Macro F1 = 0.4089, Final Metric = 0.6788
  New best metric! Saving model...
Epoch 06: Train Loss = 1.3417, Val Loss = 1.3798
  Binary F1 = 0.9539, Macro F1 = 0.4639, Final Metric = 0.7089
  New best metric! Saving model...
Epoch 07: Train Loss = 1.2047, Val Loss = 1.3088
  Binary F1 = 0.9543, Macro F1 = 0.5049, Final 

## Submission

### Reloading best model

In [ ]:
model_ensemble = []
for fold in range(5):
    model = mk_model().to(device)
    checkpoint = torch.load(f"best_model_fold{fold}.pth", map_location=device, weights_only=True)
    model.load_state_dict(checkpoint)
    model.eval()
    model_ensemble.append(model)

### Define preprocessing function

In [ ]:
def get_fillna_val_per_feature_col(df:DF) -> dict:
    return {col: 1.0 if col == 'rot_w' else 0 for col in get_feature_cols(df)}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}

    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(get_fillna_val_per_feature_col(df))
    )
    return df

def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

def add_linear_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations:Rotation = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True).astype("float32")
    df[LINEAR_ACC_COLS] = df[RAW_ACCELRATION_COLS] - gravity_sensor_frame
    return df

def add_acc_magnitude(df:DF, acc_cols:list[str], acc_mag_col_name:str) -> DF:
    return df.assign(**{acc_mag_col_name: np.linalg.norm(df.loc[:, acc_cols], axis=1)})

def add_quat_angle_mag(df:DF) -> DF:
    return df.assign(quat_rot_mag=np.arccos(df["rot_w"]) * 2)

def add_angular_velocity_features(df:DF) -> DF:
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    delta_rotations = rotations[1:] * rotations[:-1].inv()
    delta_rot_velocity = delta_rotations.as_rotvec()
    # Add extra line to avoid shape mismatch
    delta_rot_velocity = np.vstack((np.zeros((1, 3)), delta_rot_velocity))
    delta_rot_magnitude = norm(delta_rot_velocity, axis=1, keepdims=True)
    delta_rot_axes = delta_rot_velocity / (delta_rot_magnitude + EPSILON)
    df[DELTA_ROTATION_ANGULAR_VELOCITY_COLS] = delta_rot_velocity
    df[DELTA_ROTATION_AXES_COLS] = delta_rot_axes
    df["delta_rot_mag"] = delta_rot_magnitude.squeeze()

    return df

def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df

def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in tqdm(range(1, 6)):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = pd.concat(
            (
                df.drop(columns=tof_cols),
                # For some reasons, it's faster to call all the aggregation functions seperatly than agg(list of functions)
                df[tof_cols].mean(axis="columns").to_frame(tof_name + "_mean"),
                df[tof_cols].std(axis="columns").to_frame(tof_name + "_std"),
                df[tof_cols].median(axis="columns").to_frame(tof_name + "_median"),
                df[tof_cols].min(axis="columns").to_frame(tof_name + "_min"),
                df[tof_cols].max(axis="columns").to_frame(tof_name + "_max"),
            ),
            axis="columns",
        )
    return df

def add_diff_features(df:DF) -> DF:
    return pd.concat(
        (
            df,
            df
            .groupby("sequence_id", as_index=False, observed=True)
            [get_feature_cols(df)]
            .diff()
            .fillna(get_fillna_val_per_feature_col(df))
            .add_prefix("diff_")
        ),
        axis="columns",
    )

def length_normed_sequence_feat_arr(sequence: DF) -> ndarray:
    features = (
        sequence
        .loc[:, meta_data["feature_cols"]]
        .values
    )
    normed_sequence_len = meta_data["pad_seq_len"]
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def preprocess_sequence(sequence_df:pl.DataFrame) -> ndarray:
    return (
        sequence_df                     
        .to_pandas()                            # Convert to pandas dataframe.
        .pipe(imputed_features)                 # Impute missing data.
        .pipe(norm_quat_rotations)              # Norm quaternions
        .pipe(add_linear_acc_cols)              # Add gravity free acceleration.
        .pipe(add_acc_magnitude, RAW_ACCELRATION_COLS, "raw_acc_mag")
        .pipe(add_acc_magnitude, LINEAR_ACC_COLS, "linear_acc_mag")
        .pipe(add_quat_angle_mag)
        .pipe(add_angular_velocity_features)
        .pipe(rot_euler_angles)                 # Add rotation acc expressed as euler angles.
        .pipe(agg_tof_cols_per_sensor)          # Aggregate ToF columns.
        .pipe(add_diff_features)                # 
        .loc[:, sorted(meta_data["feature_cols"])]      # Retain only the usefull columns a.k.a features.
        .sub(meta_data["mean"])                 # Subtract features by their mean, std norm pt.1.
        .div(meta_data["std"])                  # Divide by Standard deviation, std norm pt.2.
        .pipe(length_normed_sequence_feat_arr)  # get feature ndarray of sequence.
        .T                                      # Transpose to swap channel and X dimensions.
    )

### Define prediction function

In [ ]:
# def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
#     """
#     Kaggle evaluation API will call this for each sequence.
#     sequence: polars DataFrame for a single sequence
#     demographics: unused in this model
#     Returns: predicted gesture string
#     """
#     df_seq = sequence.to_pandas()
#     df_demo = demographics.to_pandas()

#     df_seq = df_seq.merge(
#         df_demo,
#         on='subject',
#         how='left',
#         validate='many_to_one',
#     )
#     right_handed_mask = df_seq['handedness'] == 1
#     df_seq.loc[right_handed_mask, imu_cols] = apply_symmetry(df_seq.loc[right_handed_mask, imu_cols])

#     x_tensor = preprocess_sequence(df_seq).to(device)

#     all_outputs = []
#     with torch.no_grad():
#         for model in model_ensemble:
#             outputs = model(x_tensor).softmax(dim=-1)
#             all_outputs.append(outputs)

#     avg_outputs = torch.mean(torch.stack(all_outputs), dim=0)
#     pred_idx = torch.argmax(avg_outputs, dim=1).item()

#     return str(gesture_classes[pred_idx])

def predict(sequence: pl.DataFrame, _: pl.DataFrame) -> str:
    """
    Kaggle evaluation API will call this for each sequence.
    sequence: polars DataFrame for a single sequence
    demographics: unused in this model
    Returns: predicted gesture string
    """
    # df_seq = sequence.to_pandas()
    # df_demo = demographics.to_pandas()

    # df_seq = df_seq.merge(
    #     df_demo,
    #     on='subject',
    #     how='left',
    #     validate='many_to_one',
    # )
    # right_handed_mask = df_seq['handedness'] == 1
    # df_seq.loc[right_handed_mask, imu_cols] = apply_symmetry(df_seq.loc[right_handed_mask, imu_cols])
    # x = torch.unsqueeze(Tensor(x), dim=0).to(device)

    x_tensor = (
        torch.unsqueeze(Tensor(preprocess_sequence(sequence)), dim=0)
        .float()
        .to(device)
    )
    print(x_tensor.shape)

    all_outputs = []
    with torch.no_grad():
        for model_idx, model in enumerate(model_ensemble): # Only take the first one bc it's the only one that takes in the correct input shape
            outputs = model(x_tensor)
            all_outputs.append(outputs)

    avg_outputs = torch.mean(torch.stack(all_outputs), dim=0)
    pred_idx = torch.argmax(avg_outputs, dim=1).item()

    return str(gesture_classes[pred_idx])

### Run inference server

In [ ]:
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            join(competition_dataset_path, 'test.csv'),
            join(competition_dataset_path, 'test_demographics.csv'),
        )
    )
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)
    inference_server.run_local_gateway(
        data_paths=(
            join(competition_dataset_path, 'test.csv'),
            join(competition_dataset_path, 'test_demographics.csv'),
        )
    )

torch.Size([1, 66, 127])
torch.Size([1, 66, 127])
torch.Size([1, 66, 127])
torch.Size([1, 66, 127])
